In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
pip install scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 62.5 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.3 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
Note: y

In [7]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier

In [8]:
# Load your dataset
data = pd.read_csv('/kaggle/input/aml-with-target/all_patients_cell_counts_with_target.csv') 

In [9]:
# Convert the target column to binary classification
data['target'] = data['target'].apply(lambda x: 0 if x == 'control' else 1)

In [10]:
# Split data into features (X) and target (y)
X = data.drop(columns=['target', 'patient'])  # Adjust 'target' column name as needed
y = data['target']


In [11]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Step 1: Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [14]:
# Step 2: Define base models for binary classification
# XGBoost for binary classification
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

In [15]:
# LightGBM with binary objective
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    metric='binary_logloss',
    max_depth=8,
    num_leaves=256,  # Based on previous depth
    learning_rate=0.1,
    n_estimators=500
)

In [16]:
# LightGBM with binary objective
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    metric='binary_logloss',
    max_depth=8,
    num_leaves=256,  # Based on previous depth
    learning_rate=0.1,
    n_estimators=500
)

In [17]:
# CatBoost for binary classification
cat_model = CatBoostClassifier(
    iterations=500, 
    depth=6, 
    learning_rate=0.1, 
    random_seed=42, 
    verbose=0
)

In [18]:
# Random Forest
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)

In [19]:
# Define Neural Network model function for binary classification
def create_nn_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification with sigmoid
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [20]:
# Wrap the neural network model using SciKerasClassifier
nn_model = KerasClassifier(model=create_nn_model, epochs=50, batch_size=16, verbose=0)

In [21]:
# Step 3: Combine the models in a Voting Ensemble with soft voting
voting_model = VotingClassifier(
    estimators=[
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('cat', cat_model),
        ('lgb', lgb_model),
        ('nn', nn_model)  # Adding the neural network to the ensemble
    ],
    voting='soft'  # Use soft voting to account for prediction probabilities
)

In [22]:
# Train the voting ensemble on SMOTE-balanced data
voting_model.fit(X_train_smote, y_train_smote)

[LightGBM] [Info] Number of positive: 102, number of negative: 102
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003569 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 457
[LightGBM] [Info] Number of data points in the train set: 204, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_estimators=200,
                                                     random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=None,
                                            grow_poli...
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7cd263c54430>),
                             ('lgb',
                              LGBMClassifier(max_depth=8,
                                             metric='binary_logloss',
                                             n_estimators=500, num_leaves=256,
                                             objective='binary')),
                             ('nn',
                              KerasClassifier(batch_size=16, epochs=50, model=<function create_nn_model at 0x7cd26394c9d0>, verbose=0))],
                 voting='soft')

In [23]:
# Step 4: Evaluate the ensemble model
y_pred = voting_model.predict(X_test)
print("Binary Classification (Control as 0, Others as 1):")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Binary Classification (Control as 0, Others as 1):
Accuracy: 0.9210526315789473
              precision    recall  f1-score   support

           0       0.83      0.91      0.87        11
           1       0.96      0.93      0.94        27

    accuracy                           0.92        38
   macro avg       0.90      0.92      0.91        38
weighted avg       0.92      0.92      0.92        38

